## Set up

### Set up for Colab

In [ ]:
# For runing notebook in colab
from google.colab import drive
drive.mount('/content/drive')
import os
root_of_repository = '/content/drive/MyDrive/ADL/Project/'
os.chdir(root_of_repository)

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.4 MB/s 
     |████████████████████████████████| 7.6 MB 48.7 MB/s 
     |████████████████████████████████| 182 kB 76.4 MB/s 


### Packages

In [1]:
import torch
from transformers import BertModel, BertTokenizer, AdamW, logging
logging.set_verbosity_error()
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F

import numpy as np
import pandas as pd
import copy
import time
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [2]:
from classifier_utils import SentimentClassifier, train, test
from data_load_utils import convert_text_to_token, genDataLoader

## Data Preprocessing

In [3]:
LABEL_DICT = {'fear':0, 'neutral':1, 'sad':2, 'surprise':3, 'angry':4, 'happy':5} # Mapping label code and meaning

In [4]:
TESTING = False
train_datagen = genDataLoader('train')
val_datagen = genDataLoader('val')
test_datagen = genDataLoader('test')

Processed 2000 sentences for train
Processed 4000 sentences for train
Processed 6000 sentences for train
Processed 8000 sentences for train
Processed 10000 sentences for train
Processed 12000 sentences for train
Processed 14000 sentences for train
Processed 16000 sentences for train
Processed 18000 sentences for train
Processed 20000 sentences for train
Processed 22000 sentences for train
Processed 2000 sentences for val
Processed 4000 sentences for val
Processed 2000 sentences for test
Processed 4000 sentences for test


## Modeling

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')


### Load Pretrain Model

- `wwm` means whole word masking pretrained upon EXT dataset

In [6]:
bert-base-chinese# All pretrain models in chinese
MODELS_PATHS_UNITS = {
    'BERT': ('bert-base-chinese', 768),
    'BERT-wwm': ('hfl/chinese-bert-wwm-ext', 768),
    'RoBERTa': ('uer/chinese_roberta_L-12_H-768', 768),
    'RoBERTa-wwm': ('hfl/chinese-roberta-wwm-ext', 768),
    'RoBERTa-wwm-large': ('hfl/chinese-roberta-wwm-ext-large', 1024),
    '3L RoBERTa-wwm': ('hfl/rbt3', 768),
    '3L RoBERTa-wwm-large': ('hfl/rbtl3', 1024),
}

In [7]:
models = {}
for model_name in tqdm(MODELS_PATHS_UNITS.keys()):
    sentiment_classifier = SentimentClassifier(num_classes=6, 
                                               model_name=model_name, 
                                               pretrain_path=MODELS_PATHS_UNITS[model_name][0], 
                                               hidden_size=MODELS_PATHS_UNITS[model_name][1]).to(DEVICE)
    models[model_name] = sentiment_classifier
    gc.collect()

  0%|          | 0/7 [00:00<?, ?it/s]

### Fine-tuning

In [ ]:
# Setting up
REPLACE_EXIST = False
NUM_EPOCHS = 15
BEST_MODEL_FOLDER = 'result/model/'  # Path to save best model
TRAINING_LOGS_FOLDER = 'result/training/'  # Path to save training logs

# Fine-tune each pretrain model
for model_name in tqdm(list(models.keys())):
    print('-'*10, model_name, '-'*10)
    model_path = f'{BEST_MODEL_FOLDER}best_{model_name}.pth' if not TESTING else f'{BEST_MODEL_FOLDER}best_testing_{model_name}.pth'
    log_path = f'{TRAINING_LOGS_FOLDER}{model_name}.pickle'
    if not os.path.exists(model_path) or not os.path.exists(log_path) or REPLACE_EXIST:
        optimizer = AdamW(models[model_name].parameters(), lr=1e-5, eps=1e-8, correct_bias=False, no_deprecation_warning=True)
        training_log = train(models[model_name], model_name, train_datagen, val_datagen, optimizer, DEVICE, NUM_EPOCHS, model_path)
        training_log.to_pickle(log_path)
    else:
        print('already done')
    del models[model_name]
    gc.collect()

  0%|          | 0/7 [00:00<?, ?it/s]

---------- 3L RoBERTa-wwm ----------
already done
---------- 3L RoBERTa-wwm-large ----------
already done
---------- BERT-wwm ----------
already done
---------- RoBERTa-wwm ----------
already done
---------- RoBERTa-wwm-large ----------
already done
---------- BERT ----------


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2846 [00:00<?, ?it/s]

Epoch: 1 [1992/22768 (8.75%)]	Batch Loss: 0.605810	Avg Loss: 1.028861	
Epoch: 1 [3992/22768 (17.53%)]	Batch Loss: 0.394316	Avg Loss: 0.895036	
Epoch: 1 [5992/22768 (26.32%)]	Batch Loss: 0.532252	Avg Loss: 0.845482	
Epoch: 1 [7992/22768 (35.10%)]	Batch Loss: 0.959204	Avg Loss: 0.813364	
Epoch: 1 [9992/22768 (43.89%)]	Batch Loss: 0.561306	Avg Loss: 0.785677	
Epoch: 1 [11992/22768 (52.67%)]	Batch Loss: 0.899179	Avg Loss: 0.763997	
Epoch: 1 [13992/22768 (61.45%)]	Batch Loss: 1.355647	Avg Loss: 0.750996	
Epoch: 1 [15992/22768 (70.24%)]	Batch Loss: 0.479600	Avg Loss: 0.739999	
Epoch: 1 [17992/22768 (79.02%)]	Batch Loss: 0.265840	Avg Loss: 0.728334	
Epoch: 1 [19992/22768 (87.81%)]	Batch Loss: 0.232115	Avg Loss: 0.721392	
Epoch: 1 [21992/22768 (96.59%)]	Batch Loss: 0.223941	Avg Loss: 0.712253	
Epoch 1 finished, took 568.8s
Test set: Average loss: 0.6438, Accuracy: 3847/5000 (76.9400%), Macro F1: 74.0394%, took 38.2s


...

Epoch: 13 [1992/22768 (8.75%)]	Batch Loss: 0.000482	Avg Loss: 0.012958	
Epoch: 13 [3992/22768 (17.53%)]	Batch Loss: 0.000337	Avg Loss: 0.013117	
Epoch: 13 [5992/22768 (26.32%)]	Batch Loss: 0.001014	Avg Loss: 0.012137	
Epoch: 13 [7992/22768 (35.10%)]	Batch Loss: 0.000899	Avg Loss: 0.012994	
Epoch: 13 [9992/22768 (43.89%)]	Batch Loss: 0.165601	Avg Loss: 0.013425	
Epoch: 13 [11992/22768 (52.67%)]	Batch Loss: 0.004989	Avg Loss: 0.015717	
Epoch: 13 [13992/22768 (61.45%)]	Batch Loss: 0.001294	Avg Loss: 0.017295	
Epoch: 13 [15992/22768 (70.24%)]	Batch Loss: 0.009757	Avg Loss: 0.020253	
Epoch: 13 [17992/22768 (79.02%)]	Batch Loss: 0.004660	Avg Loss: 0.020535	
Epoch: 13 [19992/22768 (87.81%)]	Batch Loss: 0.003745	Avg Loss: 0.021076	
Epoch: 13 [21992/22768 (96.59%)]	Batch Loss: 0.000352	Avg Loss: 0.021337	
Epoch 13 finished, took 557.1s
Test set: Average loss: 1.3370, Accuracy: 3782/5000 (75.6400%), Macro F1: 72.7186%, took 38.6s


  0%|          | 0/2846 [00:00<?, ?it/s]

Epoch: 14 [1992/22768 (8.75%)]	Batch Loss: 0.000358	Avg Loss: 0.011398	
Epoch: 14 [3992/22768 (17.53%)]	Batch Loss: 0.000332	Avg Loss: 0.016204	
Epoch: 14 [5992/22768 (26.32%)]	Batch Loss: 0.005452	Avg Loss: 0.015823	
Epoch: 14 [7992/22768 (35.10%)]	Batch Loss: 0.001386	Avg Loss: 0.016660	
Epoch: 14 [9992/22768 (43.89%)]	Batch Loss: 0.000162	Avg Loss: 0.015633	
Epoch: 14 [11992/22768 (52.67%)]	Batch Loss: 0.000410	Avg Loss: 0.017237	
Epoch: 14 [13992/22768 (61.45%)]	Batch Loss: 0.249188	Avg Loss: 0.016805	
Epoch: 14 [15992/22768 (70.24%)]	Batch Loss: 0.000432	Avg Loss: 0.017048	
Epoch: 14 [17992/22768 (79.02%)]	Batch Loss: 0.001060	Avg Loss: 0.017621	
Epoch: 14 [19992/22768 (87.81%)]	Batch Loss: 0.004368	Avg Loss: 0.017865	
Epoch: 14 [21992/22768 (96.59%)]	Batch Loss: 0.001098	Avg Loss: 0.017888	
Epoch 14 finished, took 556.9s
Test set: Average loss: 1.3748, Accuracy: 3815/5000 (76.3000%), Macro F1: 72.9947%, took 38.6s


  0%|          | 0/2846 [00:00<?, ?it/s]

Epoch: 15 [1992/22768 (8.75%)]	Batch Loss: 0.000291	Avg Loss: 0.010429	
Epoch: 15 [3992/22768 (17.53%)]	Batch Loss: 0.000236	Avg Loss: 0.011677	
Epoch: 15 [5992/22768 (26.32%)]	Batch Loss: 0.000431	Avg Loss: 0.011904	
Epoch: 15 [7992/22768 (35.10%)]	Batch Loss: 0.000336	Avg Loss: 0.011115	
Epoch: 15 [9992/22768 (43.89%)]	Batch Loss: 0.001200	Avg Loss: 0.012474	
Epoch: 15 [11992/22768 (52.67%)]	Batch Loss: 0.000139	Avg Loss: 0.014724	
Epoch: 15 [13992/22768 (61.45%)]	Batch Loss: 0.002118	Avg Loss: 0.016800	
Epoch: 15 [15992/22768 (70.24%)]	Batch Loss: 0.000581	Avg Loss: 0.016642	
Epoch: 15 [17992/22768 (79.02%)]	Batch Loss: 0.244950	Avg Loss: 0.017071	
Epoch: 15 [19992/22768 (87.81%)]	Batch Loss: 0.140092	Avg Loss: 0.016527	
Epoch: 15 [21992/22768 (96.59%)]	Batch Loss: 0.001325	Avg Loss: 0.018063	
Epoch 15 finished, took 557.3s
Test set: Average loss: 1.3379, Accuracy: 3788/5000 (75.7600%), Macro F1: 73.0581%, took 38.6s
